In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
from warnings import simplefilter
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
namestocommon = geodict.namestocommon
geotogeoid = geodict.geotogeoid
tofullcensus = geodict.tofullcensus
import sqlite3 as sq

In [2]:
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
#calculate real and percent change between all columns for all possible time frames
def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames

In [23]:
data = pd.read_csv('../../Data Downloads/WP17/TN089.csv', skiprows = 3, #skipfooter = 87, 
                   engine = 'python')
total = pd.read_csv('../../Data Downloads/WP17/TN080.csv', skiprows = 3, #skipfooter = 87, 
                   engine = 'python')
pop = pd.read_csv('../../Data Downloads/WP17/TN001.csv', skiprows = 3, #skipfooter = 87, 
                   engine = 'python')

In [24]:
pop.head()

,FILE,COUNTY,TYPE,REGION,EA,MSA,CSA,MDIV,MICRO,NAME,1969,1970,1971,1972,1973,1974,1975,1976,1977,1978,1979,1980,1981,1982,1983,1984,1985,1986,1987,1988,1989,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050
0,001,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,UNITED STATES,201297.999,203982.313,206860.314,209283.987,211357.665,213341.613,215465.210,217562.808,219760.030,222095.369,224567.355,227225.622,229465.841,231664.847,233792.714,235825.529,237924.747,240133.891,242289.858,244499.828,246819.820,249622.800,252980.928,256514.223,259918.577,263125.961,266278.459,269394.261,272646.870,275854.086,279040.117,282162.376,284968.913,287625.147,290107.892,292805.252,295516.550,298379.873,301231.161,304093.924,306771.487,309346.806,311718.780,314102.549,316427.327,318906.933,321420.589,324160.750,327167.878,330206.743,333280.039,336382.525,339514.788,342676.714,345864.633,349080.798,352314.552,355561.966,358822.028,362086.877,365361.981,368644.063,371883.967,375077.921,378237.066,381357.775,384441.850,387493.632,390514.766,393507.447,396475.193,399418.767,402339.165,405237.234,408113.997,410979.552,413840.609,416695.432,419546.606,422400.687,425257.627,428118.942
1,001,0.0,2,5.0,0.0,0.0,0.0,0.0,0.0,SOUTHEAST,43440.000,44054.111,45014.187,46010.044,46981.447,47944.788,48773.728,49500.379,50298.300,51098.198,51964.133,52874.779,53627.458,54248.968,54856.480,55514.841,56199.132,56861.404,57536.634,58120.851,58733.619,59516.122,60500.666,61508.138,62530.549,63574.211,64601.937,65611.342,66655.300,67626.581,68569.381,69497.232,70317.959,71151.666,71961.591,73019.229,74148.386,75065.947,76122.849,77057.840,77814.936,78565.573,79266.618,79980.739,80622.406,81365.172,82177.832,83052.119,83997.730,84954.564,85923.375,86902.833,87893.160,88894.298,89905.387,90927.023,91956.439,92992.666,94035.410,95082.632,96135.722,97193.866,98244.087,99285.221,100320.235,101348.053,102369.065,103384.355,104394.333,105399.488,106400.716,107398.142,108392.034,109382.518,110369.824,111356.669,112344.812,113333.769,114324.209,115317.887,116314.772,117315.313
2,001,47000.0,3,5.0,0.0,0.0,0.0,0.0,0.0,TENNESSEE,3897.000,3942.198,4010.021,4086.654,4135.692,4197.782,4255.887,4323.183,4394.905,4453.517,4524.065,4600.684,4627.643,4646.041,4659.803,4686.798,4715.312,4738.710,4782.937,4822.439,4854.475,4894.492,4966.587,5049.742,5137.584,5231.438,5326.936,5416.643,5499.233,5570.045,5638.706,5703.719,5750.789,5795.918,5847.812,5910.809,5991.057,6088.766,6175.727,6247.411,6306.019,6356.585,6398.408,6455.469,6496.130,6547.779,6600.299,6664.107,6733.407,6803.366,6874.045,6945.335,7017.241,7089.760,7162.818,7236.441,7310.436,7384.700,7459.211,7533.799,7608.583,7683.485,7757.476,7830.480,7902.719,7974.108,8044.682,8114.528,8183.675,8252.168,8320.070,8387.395,8454.161,8520.387,8586.079,8651.463,8716.671,8781.649,8846.473,8911.260,8976.010,9040.766
3,001,0.0,6,0.0,0.0,0.0,174.0,0.0,0.0,"CHATTANOOGA-CLEVELAND-DALTON, TN-GA-AL",586.505,596.787,611.606,626.025,642.325,653.632,659.833,669.234,680.248,690.417,703.312,712.490,717.568,714.641,710.977,714.390,716.249,717.603,724.854,731.369,733.361,735.160,743.568,752.143,763.093,773.850,785.785,795.763,807.627,815.962,824.737,834.482,843.417,850.720,858.544,867.550,877.109,889.764,900.841,910.616,917.579,924.729,930.246,936.382,940.910,944.904,950.005,955.642,961.965,968.273,974.573,980.847,987.098,993.322,999.503,1005.649,1011.724,1017.722,1023.635,1029.434,1035.142,1040.740,1046.095,1051.196,1056.071,1060.712,1065.125,1069.323,1073.311,1077.095,1080.682,1084.078,1087.282,1090.303,1093.139,1095.821,1098.364,1100.766,1103.031,1105.179,1107.205,1109.116
4,001,0.0,6,0.0,0.0,0.0,297.0,0.0,0.0,"JACKSON-BROWNSVILLE, TN",109.768,110.173,112.181,115.270,115.729,116.692,117.618,118.743,119.909,119.671,121.247,

In [25]:
geos = ['CHEATHAM, TN', 'DAVIDSON, TN', 'DICKSON, TN', 'HOUSTON, TN', 'HUMPHREYS, TN', 'MONTGOMERY, TN', 'ROBERTSON, TN', 'RUTHERFORD, TN', 
        'STEWART, TN', 'SUMNER, TN', 'TROUSDALE, TN', 'WILLIAMSON, TN', 'WILSON, TN']
data = data.loc[data['NAME'].isin(geos)]
total = total.loc[total['NAME'].isin(geos)]
pop = pop.loc[pop['NAME'].isin(geos)]

In [26]:
data = data[['NAME', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2014', '2015', '2016', '2017', '2020', '2022', '2023', '2025', '2035', '2045']].reset_index(drop = True)
total = total[['NAME', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2014', '2015', '2016', '2017', '2020', '2022', '2023', '2025', '2035', '2045']].reset_index(drop = True)
pop = pop[['NAME', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2014', '2015', '2016', '2017', '2020', '2022', '2023', '2025', '2035', '2045']].reset_index(drop = True)

In [27]:
total.head()

,NAME,2008,2009,2010,2011,2012,2013,2014,2014,2015,2016,2017,2020,2022,2023,2025,2035,2045
0,"CHEATHAM, TN",1235.941,1197.339,1204.069,1248.342,1299.431,1303.675,1312.533,1312.533,1385.144,1373.932,1410.763,1523.058,1601.100,1640.850,1721.228,2100.557,2455.090
1,"DAVIDSON, TN",28057.775,28819.923,29351.650,29863.418,29189.669,28794.199,31608.580,31608.580,34344.025,35451.121,36359.859,39315.482,41445.944,42535.919,44805.410,56515.328,68003.482
2,"DICKSON, TN",1461.660,1454.193,1486.098,1506.676,1536.894,1537.524,1582.991,1582.991,1689.975,1748.714,1798.067,1947.427,2051.445,2104.047,2209.593,2679.866,3082.903
3,"HOUSTON, TN",208.792,209.621,224.683,231.097,233.120,226.048,232.484,232.484,235.275,241.419,248.552,271.282,287.354,295.581,312.478,390.949,458.658
4,"HUMPHREYS, TN",517.225,523.085,550.853,555.949,563.753,554.647,570.428,570.428,597.408,606.544,619.820,662.551,692.897,708.306,739.514,877.553,988.386


In [28]:
pop.to_csv('pop.csv')

PermissionError: [Errno 13] Permission denied: 'pop.csv'

In [ ]:
total.to_csv('totalpersonalincome.csv')

In [29]:
data.to_csv('pcincome.csv')